In [1]:
import pandas as pd

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,48,2024-02-08,1,118.744681,98.621277,0.554532,12.061702,27.372340,0.200000,118.921277,...,11.307843,24.458824,0.182118,121.958824,0,2,3,3,10,2024
1,51,2024-02-08,-4,112.340000,101.012000,0.529360,12.518000,22.894000,0.174060,110.872000,...,12.224000,25.868000,0.231940,113.750000,0,2,3,5,27,2024
2,51,2024-02-08,1,118.220000,99.550000,0.559360,10.706000,22.428000,0.204940,118.702000,...,11.004000,29.514000,0.208360,119.668000,0,2,3,3,7,2024
3,49,2024-02-08,6,114.770833,97.643750,0.556750,11.631250,24.970833,0.183083,117.083333,...,10.553061,25.571429,0.181449,116.726531,0,2,3,3,6,2024
4,51,2024-02-08,1,112.920000,96.968000,0.564400,13.118000,22.804000,0.222120,116.092000,...,11.002000,22.072000,0.230300,121.432000,0,2,3,4,18,2024
5,51,2024-02-08,-1,111.060000,95.582000,0.531940,10.738000,24.252000,0.185940,114.852000,...,12.462000,22.232000,0.185940,108.402000,0,2,3,5,4,2024
6,50,2024-02-08,-3,112.591837,99.710204,0.536653,13.179592,25.065306,0.198265,112.489796,...,12.306000,26.018000,0.195440,110.590000,0,2,3,4,9,2024
7,52,2024-02-08,1,117.745098,99.090196,0.545451,13.001961,28.880392,0.213000,117.627451,...,12.656000,25.084000,0.239580,119.880000,0,2,3,4,29,2024
8,52,2024-02-08,2,114.784314,95.729412,0.560098,10.543137,26.107843,0.180529,120.031373,...,12.144231,19.813462,0.217365,114.876923,0,2,3,4,8,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', show=True):
    if model_name == 'LR':
        model = joblib.load('Models/TrainedModels/LR_Model.pkl')
        scaler = joblib.load('Models/Scalers/LR.bin')
    elif model_name == 'RF':
        model = joblib.load('Models/TrainedModels/RF_Model.pkl')
        scaler = joblib.load('Models/Scalers/RF.bin')
    elif model_name == 'GB':
        model = joblib.load('Models/TrainedModels/GB_Model.pkl')
        scaler = joblib.load('Models/Scalers/GB.bin')
    elif model_name == 'SVM':
        model = joblib.load('Models/TrainedModels/SVM_Model.pkl')
        scaler = joblib.load('Models/Scalers/SVM.bin')
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    predictions = model.predict(scaled_df)
    
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-08,GSW,IND,3.050000,1.400000,0
1,2024-02-08,SAS,ORL,5.100000,1.181818,0
2,2024-02-08,DAL,NYK,1.520833,2.600000,0
3,2024-02-08,CLE,BRK,1.444444,2.850000,1
4,2024-02-08,MIN,MIL,1.980392,1.847458,0
5,2024-02-08,CHI,MEM,1.434783,2.900000,1
6,2024-02-08,DET,POR,2.800000,1.458716,0
7,2024-02-08,UTA,PHO,3.150000,1.387597,0
8,2024-02-08,DEN,LAL,1.617284,2.360000,1


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-08,GSW,IND,3.050000,1.400000,0
1,2024-02-08,SAS,ORL,5.100000,1.181818,0
2,2024-02-08,DAL,NYK,1.520833,2.600000,0
3,2024-02-08,CLE,BRK,1.444444,2.850000,1
4,2024-02-08,MIN,MIL,1.980392,1.847458,0
5,2024-02-08,CHI,MEM,1.434783,2.900000,1
6,2024-02-08,DET,POR,2.800000,1.458716,1
7,2024-02-08,UTA,PHO,3.150000,1.387597,0
8,2024-02-08,DEN,LAL,1.617284,2.360000,1


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-08,GSW,IND,3.050000,1.400000,0
1,2024-02-08,SAS,ORL,5.100000,1.181818,0
2,2024-02-08,DAL,NYK,1.520833,2.600000,0
3,2024-02-08,CLE,BRK,1.444444,2.850000,0
4,2024-02-08,MIN,MIL,1.980392,1.847458,0
5,2024-02-08,CHI,MEM,1.434783,2.900000,1
6,2024-02-08,DET,POR,2.800000,1.458716,1
7,2024-02-08,UTA,PHO,3.150000,1.387597,0
8,2024-02-08,DEN,LAL,1.617284,2.360000,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-08,GSW,IND,3.050000,1.400000,0
1,2024-02-08,SAS,ORL,5.100000,1.181818,0
2,2024-02-08,DAL,NYK,1.520833,2.600000,0
3,2024-02-08,CLE,BRK,1.444444,2.850000,0
4,2024-02-08,MIN,MIL,1.980392,1.847458,0
5,2024-02-08,CHI,MEM,1.434783,2.900000,1
6,2024-02-08,DET,POR,2.800000,1.458716,0
7,2024-02-08,UTA,PHO,3.150000,1.387597,0
8,2024-02-08,DEN,LAL,1.617284,2.360000,1
